<a href="https://colab.research.google.com/github/claramarto11/AAvanceproject/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Advanced Automation 2023's Project
# Recommendation Engine & Hit Predictor // Spotify API

!pip install requests
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
import base64
from requests import post, get
import json
import csv
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 5.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
#Authentication 

id= "011d29dea2f644f78b4f5368ba0d317e"
secret= "86b22acbd4bf498591b038d69adea504"

client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

playlist_link1 = "https://open.spotify.com/playlist/1mXeUy5gxuy8juz2oiL5H6?si=537119c577b7402c"
playlist_id1 = playlist_link1.split("/")[-1].split("?")[0]

playlist_link0 = "https://open.spotify.com/playlist/0taMcedcWUKNvRKicX09ZX?si=2c702c6b8a3d44b1"
playlist_id0 = playlist_link0.split("/")[-1].split("?")[0]


In [3]:
def call_playlist(playlist_id):
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    playlist = sp.playlist_tracks(playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)     
    return playlist_df


In [5]:
df1=call_playlist(playlist_id1)
df1.to_csv('Playlist_joao_1.csv')
df1

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,The Beatles,Abbey Road (Remastered),Something - Remastered 2009,0pNeVovbiZHkulpGeOx1Gj,0.396,0.338,0,-10.872,1,0.0304,0.000003,0.1380,0.377,133.522,182293,4
1,The Beatles,Abbey Road (Remastered),Oh! Darling - Remastered 2009,2mxByJWOajjiVsLWjNXvDJ,0.422,0.656,4,-8.412,1,0.0397,0.027900,0.2680,0.549,173.930,207240,3
2,The Beatles,Abbey Road (Remastered),Octopus's Garden - Remastered 2009,0suLngfo7rJoetk7Ub6N8l,0.626,0.512,1,-9.150,0,0.0247,0.000046,0.2150,0.730,92.225,170720,4
3,The Beatles,Abbey Road (Remastered),I Want You (She's So Heavy) - Remastered 2009,3Z25k4ZF6QENy2d9YatsM5,0.370,0.515,2,-9.487,0,0.0321,0.205000,0.0966,0.420,115.732,467320,4
4,The Beatles,Yellow Submarine (Remastered),Yellow Submarine - Remastered 2009,3oEo8Pqm5IAi8wQfCI5BpR,0.605,0.536,1,-10.069,1,0.0421,0.000000,0.5280,0.688,110.849,159720,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,IDLES,Brutalism,Mother,1nGJuMCIMDeOAy0MAzKpQC,0.317,0.839,1,-6.272,0,0.0509,0.003040,0.2090,0.293,199.355,207145,4
96,IDLES,Joy as an Act of Resistance.,Great,1c1tdTuWxxTm1CgbdpLtZG,0.397,0.987,1,-3.269,1,0.1470,0.000002,0.1050,0.237,108.251,164133,4
97,IDLES,Joy as an Act of Resistance.,Love Song,4VucS4cgEjZg0AWncmvU0K,0.268,0.827,1,-4.446,1,0.0478,0.012200,0.1090,0.144,168.442,185166,4
98,The Goo Goo Dolls,Dizzy up the Girl,Iris,6Qyc6fS4DsZjB2mRW9DsQs,0.315,0.715,11,-8.072,0,0.0362,0.000008,0.0942,0.497,155.925,289533,3


In [6]:
df0=call_playlist(playlist_id0)
df0.to_csv('Playlist_joao_0.csv')
df0

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Noel Gallagher's High Flying Birds,Chasing Yesterday,In the Heat of the Moment,0oq87CnmjoZnZFkNB2BUYG,0.547,0.828,6,-4.338,1,0.0282,0.000123,0.415,0.464,105.916,209480,4
1,Masego,Lady Lady,Tadow,51rPRW8NjxZoWPPjnRGzHw,0.704,0.487,9,-6.407,0,0.0604,0.000086,0.111,0.313,121.726,301893,4
2,FKJ,Risk,Risk,0sW9jZgbLiWxqdFagmZ9fM,0.761,0.359,7,-12.326,1,0.0703,0.000143,0.147,0.505,103.069,238835,4
3,Phum Viphurit,"Hello, Anxiety (STUTS Remix)","Hello, Anxiety (STUTS Remix)",7Ie7xLLlektwxEmvHzuBar,0.790,0.854,6,-6.803,0,0.0384,0.095000,0.108,0.647,125.974,266071,4
4,jaymeso,Quando Ela Passa,Quando Ela Passa,4TR7xfTa41Rbq9pswfE1z8,0.799,0.490,1,-5.596,0,0.1410,0.002290,0.109,0.656,140.009,138897,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Carlos Do Carmo,Fado Maestro,Os Putos,2UiVYBid1vQKfIs4NpM14J,0.689,0.354,6,-9.921,1,0.0975,0.000033,0.322,0.654,105.797,206467,4
96,The Legendary Tigerman,Femina (Deluxe Edition),These Boots Are Made For Walkin',2G35I5lsee9bQYsevngYdS,0.767,0.624,11,-8.104,0,0.0435,0.105000,0.298,0.360,124.026,221653,4
97,Sun Kil Moon,Common As Light And Love Are Red Valleys Of Blood,I Love Portugal,4tvu7JgzWJuVnF2XhYWtGo,0.618,0.356,9,-10.211,1,0.0326,0.039200,0.126,0.594,102.029,478147,4
98,The White Stripes,Elephant,Seven Nation Army,47wPvRG8FEwbZP22UBgTQr,0.741,0.469,4,-7.627,0,0.0805,0.447000,0.306,0.313,123.904,231920,4
